In [ ]:
base=

In [1]:
# Improved Flood Detection Model for South Africa
# Based on DeepMind's starter notebook with enhancements

import os
import math
import collections
from collections.abc import Callable, Sequence
from functools import partial
from typing import Any

# Data processing
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d

# Deep Learning
import jax
import jax.numpy as jnp
import flax
from flax import linen as nn
from flax.training import train_state
import optax

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Set random seed for reproducibility
SEED = 42
rng = np.random.default_rng(seed=SEED)

# Constants
BASE_PATH = 'Zindi_comp'  # Update this path to your data directory
NUM_CLASSES = 2
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM = (128, 128, len(BAND_NAMES))

class TimeSeriesPreprocessor:
    """Enhanced time series preprocessing with advanced feature engineering."""
    def __init__(self, window_sizes=[3, 7, 14, 30]):
        self.window_sizes = window_sizes
    
    def add_rolling_features(self, data: np.ndarray) -> np.ndarray:
        """Add rolling window statistics for precipitation."""
        features = [data]
        
        for window in self.window_sizes:
            # Rolling mean
            rolling_mean = np.apply_along_axis(
                lambda x: np.convolve(x, np.ones(window)/window, mode='same'),
                axis=1, arr=data)
            features.append(rolling_mean)
            
            # Rolling max
            rolling_max = np.apply_along_axis(
                lambda x: np.maximum.accumulate(x, axis=0),
                axis=1, arr=rolling_mean)
            features.append(rolling_max)
            
            # Rolling std
            rolling_std = np.apply_along_axis(
                lambda x: np.std(x[max(0, len(x)-window):]),
                axis=1, arr=data)
            features.append(np.expand_dims(rolling_std, -1))
        
        # Add cumulative precipitation
        cumsum = np.cumsum(data, axis=1)
        features.append(cumsum)
        
        # Add smoothed signal
        smoothed = gaussian_filter1d(data, sigma=2.0, axis=1)
        features.append(smoothed)
        
        return np.concatenate(features, axis=-1)
    
    def transform(self, timeseries: np.ndarray) -> np.ndarray:
        """Transform time series data with all features."""
        enhanced_features = self.add_rolling_features(timeseries)
        
        # Normalize features
        mean = np.mean(enhanced_features, axis=(0, 1), keepdims=True)
        std = np.std(enhanced_features, axis=(0, 1), keepdims=True)
        normalized = (enhanced_features - mean) / (std + 1e-8)
        
        return normalized.astype(np.float32)

def sigmoid_focal_loss(logits, labels, alpha=0.25, gamma=2.0):
    """Compute focal loss for binary classification."""
    p = jnp.clip(jax.nn.sigmoid(logits), 1e-6, 1-1e-6)
    ce_loss = -(labels * jnp.log(p) + (1 - labels) * jnp.log(1 - p))
    p_t = labels * p + (1 - labels) * (1 - p)
    focal_weight = jnp.power(1 - p_t, gamma)
    alpha_weight = labels * alpha + (1 - labels) * (1 - alpha)
    return focal_weight * alpha_weight * ce_loss

def load_and_preprocess_data():
    """Load and preprocess all data."""
    # Load CSV files
    train_path = os.path.join(BASE_PATH, 'Train.csv')
    test_path = os.path.join(BASE_PATH, 'Test.csv')
    
    if not os.path.exists(train_path) or not os.path.exists(test_path):
        raise FileNotFoundError(f"Data files not found in {BASE_PATH}")
    
    data = pd.read_csv(train_path)
    data_test = pd.read_csv(test_path)
    
    # Process event IDs
    data['event_id'] = data['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    data['event_idx'] = data.groupby('event_id', sort=False).ngroup()
    data_test['event_id'] = data_test['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    data_test['event_idx'] = data_test.groupby('event_id', sort=False).ngroup()
    
    # Create time steps
    data['event_t'] = data.groupby('event_id').cumcount()
    data_test['event_t'] = data_test.groupby('event_id').cumcount()
    
    return data, data_test

# Add model architecture classes here...

def main():
    """Main execution function."""
    print("Loading data...")
    data, data_test = load_and_preprocess_data()
    
    print("Preprocessing time series...")
    preprocessor = TimeSeriesPreprocessor()
    # Add preprocessing steps...
    
    print("Training model...")
    # Add training steps...
    
if __name__ == "__main__":
    main()

Loading data...


FileNotFoundError: Data files not found in Zindi_comp